In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
import time

In [2]:
person_list = [
#     #BTC
    "saylor", "elonmusk", "tyler", "cameron", "BarrySilbert", "novogratz", "APompliano", "SatoshiLite",
    "TimDraper", "brian_armstrong", "aantonop", "naval", "justinsuntron", "cz_binance", "lopp",
    "scottmelker", "TheCryptoDog", "BTC_JackSparrow", "VentureCoinist", "rektcapital",
    "Rager", "PeterMcCormack", "adam3us", "PeterLBrandt", "balajis", 
]

num_tweets_per_tag = 1000

In [3]:
# Keywords
keywords = [ 
    #BITCOIN keywords
    "#bitcoin", "#btc", "#bitcoinprice", "#buybitcoin", "#bitcoincash", "#btcusd", "#btcusdt", "$btc", "btc", "bitcoin",
]

In [4]:
def scrape_data_user(twitter_user, keywords, 
                     start_date="2020-01-01", end_date="2021-12-31", num_tweets_per_tag=100):
    df = pd.DataFrame()
    start = time.time()  
    for word in keywords:
        try:
            df = df.append(pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
                f'{word} from:{twitter_user} lang:en since:{start_date} until:{end_date}').get_items(), num_tweets_per_tag)))
        except Exception as e:
            print("An error occured.\n")
            continue
    
    print(f'Number of tweets from {twitter_user} : {df.shape}')
    print(df.columns)
    
    
    df_ = None
    if df.shape[0] > 0:
        df["username"] = df["user"].apply(lambda x: x["username"])
        df_ = df[["id", "url", "date", "content", "username", "replyCount", "retweetCount", "likeCount", "quoteCount", "hashtags"]]
        
    
    print(f'Time taken : {((time.time() - start)/60):.1f} mins')
    return df_

In [5]:
# Initializing Dictionary of DataFrames for Each of the 23 Countries
personsDf = {}

In [6]:
final_df = pd.DataFrame()
for person in person_list:
    personsDf[person] = scrape_data_user(person, keywords)
    final_df = final_df.append(personsDf[person])
    
final_df = pd.DataFrame(final_df)

Number of tweets from saylor : (251, 27)
Index(['url', 'date', 'content', 'renderedContent', 'id', 'user', 'replyCount',
       'retweetCount', 'likeCount', 'quoteCount', 'conversationId', 'lang',
       'source', 'sourceUrl', 'sourceLabel', 'outlinks', 'tcooutlinks',
       'media', 'retweetedTweet', 'quotedTweet', 'inReplyToTweetId',
       'inReplyToUser', 'mentionedUsers', 'coordinates', 'place', 'hashtags',
       'cashtags'],
      dtype='object')
Index(['Columnsurl', 'Columnsdate', 'Columnscontent', 'ColumnsrenderedContent',
       'Columnsid', 'Columnsuser', 'ColumnsreplyCount', 'ColumnsretweetCount',
       'ColumnslikeCount', 'ColumnsquoteCount', 'ColumnsconversationId',
       'Columnslang', 'Columnssource', 'ColumnssourceUrl',
       'ColumnssourceLabel', 'Columnsoutlinks', 'Columnstcooutlinks',
       'Columnsmedia', 'ColumnsretweetedTweet', 'ColumnsquotedTweet',
       'ColumnsinReplyToTweetId', 'ColumnsinReplyToUser',
       'ColumnsmentionedUsers', 'Columnscoordinates',

KeyboardInterrupt: 

In [8]:
final_df = final_df.drop_duplicates(subset=["id"])
print(final_df.groupby("username").size())

username
APompliano         148
BTC_JackSparrow    351
BarrySilbert        94
PeterLBrandt       140
PeterMcCormack     231
Rager              215
SatoshiLite         62
TimDraper           80
VentureCoinist     205
aantonop           196
adam3us            244
balajis            162
cameron            113
cz_binance         107
elonmusk             6
justinsuntron      150
lopp               168
naval               26
novogratz          143
rektcapital        213
saylor             119
scottmelker        245
tyler              137
dtype: int64


In [10]:
# Cleaning Data
df_indexes_v2 = []
user_dict = {}
for i in range(len(final_df)):
    tweet = final_df["content"].iloc[i]
    
    # To remove tweets that have more hashtags than normal text
    word_list = tweet.lower().split()
    num_normal = 0
    num_tags = 0
    for j in range(len(word_list)):
        temp = word_list[j]
        if temp[0] == '#':
            num_tags += 1
        else:
            num_normal += 1
    if num_tags < num_normal:
        df_indexes_v2.append(i)
    

df_v2 = final_df.iloc[df_indexes_v2]

#Remove tweets that have 0 likes, retweets, or comments
# df_v2 = df_v2.drop(df_v2[(df_v2["likeCount"] == 0) | (df_v2["replyCount"] == 0) | (df_v2["retweetCount"] == 0)].index)
df_v2 = df_v2[(df_v2["retweetCount"] > 0) & (df_v2["replyCount"] > 0) & (df_v2["likeCount"] > 0)]
# df_v2 = df_v2.iloc[indices]
print(f'Shape of df after cleaning: {df_v2.shape}')
# print(len(indices))

Shape of df after cleaning: (2757, 10)


In [11]:
df_v2.shape

(2757, 10)

In [12]:
df_v2["username"].unique()

array(['saylor', 'elonmusk', 'tyler', 'cameron', 'BarrySilbert',
       'novogratz', 'APompliano', 'SatoshiLite', 'TimDraper', 'aantonop',
       'naval', 'justinsuntron', 'cz_binance', 'lopp', 'scottmelker',
       'BTC_JackSparrow', 'VentureCoinist', 'rektcapital', 'Rager',
       'PeterMcCormack', 'adam3us', 'PeterLBrandt', 'balajis'],
      dtype=object)

In [13]:
df_v2.reset_index(drop=True).to_csv("btc_userwise_2020.csv")